In [1]:
pip install --upgrade MetaTrader5

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from scipy.signal import find_peaks
import MetaTrader5 as mt5

In [3]:
# Initialize MetaTrader 5 connection
if not mt5.initialize():
    print("Initialization failed")
    quit()

DEFINING FUNCTIONS

In [4]:
#DROP THE UNNCESSARY COLUMNS IN THE DATAFRAME AND CONVERTING THE TIME TO HUMAN READABLE FORMAT
def df_convert_hr(rates_frame):
    rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    rates_frame.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    rates_frame["time"] = pd.to_datetime(rates_frame["time"])
    return rates_frame

#Detect the entry drills
def sell_entry_drill():
    pass

def buy_entry_drill():
    pass

DOWNLOAD THE OHLC DATA FROM THE LAST 7 DAYS

In [11]:
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1

#GETTING THE TIMESTAMP FOR TODAY AND 3 DAYS AGO
utc_from = datetime.now(timezone.utc) - timedelta(days=7)
utc_to = datetime.now(timezone.utc)
startdate=utc_from.strftime('%Y%m%d')
enddate=utc_to.strftime('%Y%m%d')

# Get the OHLC data
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
# Check if the data is downloaded successfully
if rates is None:
    print("No data available, error code =", mt5.last_error())
else:
    # Convert the data to a pandas DataFrame
    rates_frame = pd.DataFrame(rates)
    df_convert_hr(rates_frame)


In [12]:
rates_frame

,time,open,high,low,close
0,2024-12-30 00:01:00,1.04251,1.04257,1.04246,1.04253
1,2024-12-30 00:02:00,1.04246,1.04270,1.04246,1.04267
2,2024-12-30 00:03:00,1.04262,1.04262,1.04261,1.04262
3,2024-12-30 00:04:00,1.04283,1.04303,1.04283,1.04302
4,2024-12-30 00:05:00,1.04262,1.04284,1.04262,1.04283
...,...,...,...,...,...
5753,2025-01-03 23:54:00,1.03082,1.03082,1.03079,1.03080
5754,2025-01-03 23:55:00,1.03079,1.03079,1.03071,1.03078
5755,2025-01-03 23:56:00,1.03078,1.03089,1.03077,1.03078
5756,2025-01-03 23:57:00,1.03078,1.03080,1.03071,1.03080


DETERMINE THE MOST RECENT HIGHEST/LOWEST POINTS and THE TREND FROM THE LAST 35 MINS 

In [13]:
#Determine the max and min value of the highs and lows for the last 35 mins
Last_35 = rates_frame.tail(35)
MaxHighPrice = Last_35["high"].max()
MinLowPrice = Last_35["low"].min()

#Determine the if the last 35 mins is in a downtrend or uptrend
HighestPoint = rates_frame[rates_frame["high"] == MaxHighPrice].iloc[-1]
LowestPoint = rates_frame[rates_frame["low"] == MinLowPrice].iloc[-1]

if HighestPoint["time"] > LowestPoint["time"]:
    trend_35 = "uptrend"
elif HighestPoint["time"] < LowestPoint["time"]:
    trend_35 = "downtrend"
else:
    trend_35 = "sideways"

DETERMINE THE SESSION LOW/HIGH OF PRICE

In [14]:
date_today = date.today()

#HAVE THE DATAFRAME FOR THE LATEST LONDON SESSION:
ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 9) & (rates_frame['time'].dt.hour < 12)]
#HAVE THE DATAFRAME FOR THE LATEST NEW YORK SESSION: 
ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 12) & (rates_frame['time'].dt.hour < 19)]